In [ ]:
#| default_exp basic

In [ ]:
#| export
from __future__ import annotations

# basic
> basic helpers

# Prologue


In [ ]:
#| export
import pprint
from inspect import Parameter
from typing import Hashable
from typing import Iterable
from typing import Mapping
from typing import Self
from typing import Type
from typing import TypeAlias
from typing import TypeVar

import fastcore.all as FC


In [ ]:
from pathlib import Path
from types import NoneType

from fastcore.test import *


In [ ]:
from olio.test import *


----

In [ ]:
#| export

_EMPTY: TypeAlias = Parameter.empty
EmptyT = Type[_EMPTY]


# AD

In [ ]:
#| export

_VT = TypeVar('_VT')
# from `fastcore` + generics
class AD(dict[str, _VT]):
    "`dict` subclass that also provides access to keys as attrs"
    def __getattr__(self, k:str) -> _VT: return self[k] if k in self else FC.stop(AttributeError(k))  # type: ignore
    def __setattr__(self, k, v:_VT): (self.__setitem__, super().__setattr__)[k[0]=='_'](k,v)
    def __dir__(self) -> Iterable[str]: return super().__dir__() + list(self.keys())  # type: ignore
    def _repr_markdown_(self): return f'```json\n{pprint.pformat(self, indent=2)}\n```'
    def copy(self) -> Self: return type(self)(**self)


# is_listy
> Test whether `x` is list-like


In [ ]:
#| export

def is_listy(x):
    return isinstance(x, Iterable) and not isinstance(x, (bytes, str))

def is_listy_type(x):
    return issubclass(x, Iterable) and not issubclass(x, (bytes, str))


In [ ]:

for _ in (
    [1, 2, 3], (1, 2, 3), {1, 2, 3}, 
    {'a': 1}, range(3), (i for i in range(3)), FC.L(1, 2, 3),
    ):
    test_is(is_listy(_), True)
for _ in (
    'a', b'a', r'a', None,
    ):
    test_is_not(is_listy(_), True)

for _ in (list, tuple, set, dict, range, type(i for i in range(3)), FC.L):
    test_is(is_listy_type(_), True)
for _ in (str, bytes, type(r'a'), NoneType,):
    test_is_not(is_listy_type(_), True)

# flatten


In [ ]:
#| export

def flatten(o):
    "Concatenate all collections and items as a generator"
    for item in o:
        if not is_listy(item): yield item; continue
        try: yield from flatten(item)
        except TypeError: yield item


In [ ]:
for _, expected in (
    ([], []),  
    ([1, 2, 3], (1, 2, 3)), 
    ((1, (2, 3)), (1, 2, 3)), 
    ([('a', (2,)), ('c', (4,))], ('a', 2, 'c', 4)), 
):
    test_eq(flatten(_), expected)



# shorten
> truncate string


In [ ]:
#| export

def shorten(x, mode='l', limit=40, trunc='…', empty='') -> str:
    s = str(x)
    if len(s) > limit:
        # s = f'{s[:limit//2]} ... {s[:-limit//2]}'
        # s = s[:limit] + '..' * (len(s) > limit)
        # s = (
        #         f'...{s[sl1:sl2]}'
        #         if mode == 'l' else 
        #         f's[sl1:sl2]...'
        #     )
        # sl1, sl2, l, r = (-limit, None, trunc, empty) if mode == 'l' else (None, limit, empty, trunc)
        # s = f'{l}{s[sl1:sl2]}{r}'
        l, m, r = (
            (empty, trunc, s[-limit:]) if mode == 'l' else 
            (s[:limit], trunc, empty) if mode == 'r' else 
            (s[:(limit//2)-1], f" {trunc} ", s[-(limit//2-1):])
        )
        s = f'{l}{m}{r}'
    return s


In [ ]:
test_eq(f"{shorten(0)}", '0')
test_eq(f"{shorten(234)}", '234')
test_eq(f"{shorten('asdfgh')}", 'asdfgh')
test_eq(f"{shorten('It was the best of times', limit=12)}", '…est of times')
test_eq(f"{shorten('ad2663b4-5ff5-40e3-a6ed-cc35f5627f8d', limit=17)}", '…a6ed-cc35f5627f8d')
test_eq(f"{shorten('ad2663b4-5ff5-40e3-a6ed-cc35f5627f8d', 'r', limit=17)}", 'ad2663b4-5ff5-40e…')
test_eq(f"{shorten('ad2663b4-5ff5-40e3-a6ed-cc35f5627f8d', 'c', limit=17)}", 'ad2663b … 5627f8d')


# Dict helpers


## pops_

In [ ]:
#| export
def pops_(d: dict, *ks: Hashable) -> dict:
    "Pop existing `ks` items from `d` in-place into a dictionary."
    return {k:d.pop(k) for k in ks if k in d}


In [ ]:
test_eq(pops_({'a': 1, 'b': 2, 'c': 3}, 'a', 'b'), {'a': 1, 'b': 2})
test_eq(pops_({'a': 1, 'b': 2, 'c': 3}, 'd'), {})
test_eq(pops_({'a': 1, 'b': 2, 'c': 3}, 'a', 'c', 'd'), {'a': 1, 'c': 3})
test_eq(pops_({}, 'a'), {})
test_eq(pops_({'a': 1}, 'a', 'a'), {'a': 1})


## pops_values_

In [ ]:
#| export
def pops_values_(d: dict, *ks: Hashable) -> tuple:
    "Pop existing `ks` items from `d` in-place into a tuple of values or `Parameter.empty` for missing keys."
    return tuple(d.pop(k, Parameter.empty) for k in ks)


In [ ]:
test_eq(pops_values_({'a': 1, 'b': 2, 'c': 3}, 'a', 'b'), (1, 2))
test_eq(pops_values_({'a': 1, 'b': 2, 'c': 3}, 'd'), (Parameter.empty,))
test_eq(pops_values_({'a': 1, 'b': 2, 'c': 3}, 'a', 'c', 'd'), (1, 3, Parameter.empty))
test_eq(pops_values_({}, 'a'), (Parameter.empty,))
test_eq(pops_values_({'a': 1}, 'a', 'a'), (1, Parameter.empty))


## gets

In [ ]:
#| export

def gets(d: Mapping, *ks: Hashable):
    "Fetches `ks` values, or `Parameter.empty` for missing keys, from `d` into a tuple."
    return tuple(d.get(k, Parameter.empty) for k in ks)  # type: ignore


In [ ]:
test_eq(gets({'a': 1, 'b': 2}, 'a', 'c', 'b'), (1, Parameter.empty, 2))
test_eq(gets({'a': 1, 'b': 2}), ())
a, b = gets({'a': 1, 'b': 2}, 'b', 'a')
test_eq((a, b), (2, 1))

d = {Path('a'): 1, Path('b'): 2}
test_eq(gets(d, Path('a'), Path('c'), Path('b')), (1, Parameter.empty, 2))


## update_


In [ ]:
#| export

def update_(d:dict|None=None, /, empty_value=None, **kwargs):
    "Update `d` in-place with `kwargs` whose values aren't `empty_value`"
    d = d if d is not None else {}
    for k, v in kwargs.items():
        if v is not empty_value: d[k] = v
    return d


Helper for conditionally updating dictionaries.


In [ ]:
d = {'a': 1}
update_(d, b=2, c=None)
test_eq(d, {'a': 1, 'b': 2})

d = {}
update_(d, a=True, b=Parameter.empty, empty_value=Parameter.empty)
test_eq(d, {'a': True})

d = update_(a=1, b=None)
test_eq(d, {'a': 1})


# Colophon
----

In [ ]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [ ]:
if FC.IN_NOTEBOOK:
    nb_path = '00_basic.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
